<a href="https://colab.research.google.com/github/adamserag1/Interpretability-for-VRDU-models/blob/remove-widgets-for-github-viewing/finetuning/LayoutLM3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Layout LMv3

## Imports

In [ ]:
# UNCOMMNET FOR USE IN COLAB
# !git clone https://github.com/adamserag1/Interpretability-for-VRDU-models.git

In [ ]:
!pip install datasets seqeval evaluate transformers torch

In [ ]:
from transformers import AutoProcessor, LayoutLMv3ForTokenClassification, set_seed
from PIL import Image,ImageDraw, ImageFont
from datasets import load_dataset
import torch
import pandas as pd
import evaluate
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

set_seed(0)

## Data

In [ ]:
funsd = load_dataset("nielsr/funsd", trust_remote_code=True)
labels = funsd["train"].features["ner_tags"].feature.names
id2label = {v: k for v, k in enumerate(labels)}
label2id = {v: k for k, v in enumerate(labels)}
print(id2label)
print(label2id)

In [ ]:
processor = AutoProcessor.from_pretrained("microsoft/layoutlmv3-base", apply_ocr=False)

In [ ]:
def encode(example):
  images = [Image.open(path).convert("RGB") for path in example["image_path"]]
  words = example["words"]
  boxes = example["bboxes"]
  labels = example["ner_tags"]
  encoding = processor(images,
                       words,
                       boxes=boxes,
                       word_labels=labels,
                       padding="max_length",
                       truncation=True,
                       return_tensors="pt")
  return encoding

In [ ]:
train_dataset = funsd["train"].map(encode, batched=True, remove_columns=funsd["train"].column_names)
val_dataset = funsd["test"].map(encode, batched=True, remove_columns=funsd["train"].column_names)

## Finetuning

In [ ]:
metric = evaluate.load("seqeval")
import numpy as np
def compute_metrics(p):
  predictions, labels = p
  predictions = np.argmax(predictions, axis=2)

  true_preds = [
      [id2label[p] for (p, l) in zip(prediction, label) if l != -100]
      for prediction, label in zip(predictions, labels)
  ]
  true_labels = [
      [id2label[l] for (p, l) in zip(prediction, label) if l != -100]
      for prediction, label in zip(predictions, labels)
  ]
  results = metric.compute(predictions=true_preds, references=true_labels)

  return {
      "precision": results["overall_precision"],
      "recall": results["overall_recall"],
      "f1": results["overall_f1"],
      "accuracy": results["overall_accuracy"],
  }

In [ ]:
from huggingface_hub import login
login()

In [ ]:
from transformers import TrainingArguments, Trainer
output_dir = "./layoutlmv3-finetuned-funsd"
model = LayoutLMv3ForTokenClassification.from_pretrained("microsoft/layoutlmv3-base", num_labels=len(labels), id2label=id2label, label2id=label2id)
training_args = TrainingArguments(
    output_dir=output_dir,
    max_steps=1000,
    learning_rate=1e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    eval_strategy="steps",
    seed=42,
    optim="adamw_torch",
    adam_beta1=0.9,
    adam_beta2=0.999,
    adam_epsilon=1e-08,
    lr_scheduler_type="linear",
    eval_steps=100,
    load_best_model_at_end=True,
    metric_for_best_model="eval_f1"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=processor,
    compute_metrics=compute_metrics,
)

trainer.train()

In [ ]:
'''
Model available on huggingface hub: https://huggingface.co/adamadam111/layoutlmv3-finetuned-funsd
'''

## Inference
Simple inference step for show

In [ ]:
model = LayoutLMv3ForTokenClassification.from_pretrained("adamadam111/layoutlmv3-finetuned-funsd")
processor = AutoProcessor.from_pretrained("adamadam111/layoutlmv3-finetuned-funsd")

In [ ]:
sample = funsd["test"][0]
image = Image.open(sample["image_path"]).convert("RGB")
encode_sample = processor(image, sample["words"], boxes=sample["bboxes"], word_labels=sample['ner_tags'], padding='max_length', return_tensors="pt")
outputs = model(**encode_sample)

In [ ]:
def unnormalize_box(bbox, width, height):
     return [
         width * (bbox[0] / 1000),
         height * (bbox[1] / 1000),
         width * (bbox[2] / 1000),
         height * (bbox[3] / 1000),
     ]

In [ ]:
preds = outputs.logits.argmax(-1).squeeze().tolist()
token_boxes = encode_sample.bbox.squeeze().tolist()

width, height = image.size

true_preds = [id2label[pred] for pred in preds if pred != -100]
true_boxes = [unnormalize_box(box, width, height) for box in token_boxes if box[0] != -100]

In [ ]:
draw = ImageDraw.Draw(image)
font = ImageFont.load_default()

def iob_to_label(label):
  label = label[2:]
  if not label:
    return 'other'
  return label

label2color = {'question':'blue', 'answer':'green', 'header':'orange', 'other':'violet'}

for prediction, box in zip(true_preds, true_boxes):
    predicted_label = iob_to_label(prediction).lower()
    draw.rectangle(box, outline=label2color[predicted_label])
    draw.text(
        (box[0]+10, box[1]-10), text=predicted_label, fill=label2color[predicted_label], font=font
    )

image